In [1]:
# import os
# from huggingface_hub import InferenceClient
# from datetime import datetime

# client = InferenceClient(
#     api_key=os.environ["HF_TOKEN"],
# )

# # System prompt for pseudo code generation
# SYSTEM_PROMPT = """You are an expert pseudo code generator. Your task is to:
# 1. Analyze control narrative documents
# 2. Extract key processes and workflows
# 3. Convert them into clear, well-structured pseudo code
# 4. Use proper indentation and logical flow
# 5. No actual code syntax - only pseudo code notation
# 6. No explanations or additional text - only the pseudo code
# 7. And no extras, just pseudo code based on the given information."""

# user_input = "Write pseudo code based on this control narrative document:"

# # Convert bytes to string
# full_text_str = "\n".join([text.decode("utf-8") if isinstance(text, bytes) else text for text in full_text])

# # print("\n" + "="*80)
# # print("PSEUDO CODE GENERATION - HuggingFace API")
# # print("="*80)
# # print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
# # print(f"Model: openai/gpt-oss-20b")
# # print("="*80 + "\n")

# completion = client.chat.completions.create(
#     model="openai/gpt-oss-20b",
#     messages=[
#         {
#             "role": "user",
#             "content": SYSTEM_PROMPT + "\n\n" + user_input + "\n\n" + full_text_str
#         }
#     ],
#     max_tokens=2048,
#     temperature=0.7,
# )

# response_text = completion.choices[0].message.content

# print(response_text)


In [2]:
import pymupdf

doc = pymupdf.open("document.pdf")

In [3]:
text_blocks = []
image_blocks = []

In [7]:
def extract_pdf_content(pdf_path):
    doc = pymupdf.open(pdf_path)

    text_collection = []
    image_collection = []

    for page_num, page in enumerate(doc):
        page_dict = page.get_text("dict")

        for block_id, block in enumerate(page_dict["blocks"]):
            if block["type"] == 0:
                text = " ".join(
                    span["text"]
                    for line in block["lines"]
                    for span in line["spans"]
                ).strip()

                if text:
                    text_collection.append({
                        "id": f"text_p{page_num}_b{block_id}",
                        "content": text,
                        "metadata": {
                            "page": page_num,
                            "bbox": block["bbox"],
                            "modality": "text"
                        }
                    })
        
        for img_id, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base = doc.extract_image(xref)

            image_collection.append({
                "id": f"image_p{page_num}_{img_id}",
                "image_bytes": base["image"],
                "format": base["ext"],
                "metadata": {
                    "page": page_num,
                    "xref": xref,
                    "modality": "image"
                }
            })
    return text_collection, image_collection

In [8]:
texts, images = extract_pdf_content("document.pdf")

print(len(texts), "text chunks")
print(len(images), "images")


845 text chunks
46 images
